In [1]:
import pandas as pd
from datetime import datetime

price_data = pd.read_csv(r"C:\Users\dolly\Downloads\Nat_Gas.csv")
price_data['Dates'] = pd.to_datetime(price_data['Dates'])
price_data.set_index('Dates', inplace=True)
price_data['Prices'] = price_data['Prices'].astype(float)

# Pricing function
def price_storage_contract(
    price_data,
    injection_dates,
    withdrawal_dates,
    injection_rate,
    withdrawal_rate,
    max_storage_volume,
    storage_cost_per_month
):
    storage_volume = 0.0
    total_cost = 0.0
    total_revenue = 0.0
    last_month = None

    # Combine and sort all relevant dates
    all_dates = sorted(set(injection_dates + withdrawal_dates), key=lambda x: pd.to_datetime(x))

    for date_str in all_dates:
        date = pd.to_datetime(date_str)
        if date not in price_data.index:
            continue  # Skip if date not in price data

        price = price_data.loc[date, 'Prices']

        # Inject gas
        if date_str in injection_dates:
            inject_volume = min(injection_rate, max_storage_volume - storage_volume)
            total_cost += inject_volume * price
            storage_volume += inject_volume

        # Withdraw gas
        if date_str in withdrawal_dates:
            withdraw_volume = min(withdrawal_rate, storage_volume)
            total_revenue += withdraw_volume * price
            storage_volume -= withdraw_volume

        # Apply monthly storage cost
        if last_month is None or date.month != last_month:
            total_cost += storage_cost_per_month
            last_month = date.month

    contract_value = total_revenue - total_cost
    return round(contract_value, 2)

# Sample test inputs 1
injection_dates = ['1/31/22', '2/28/22', '3/31/22']
withdrawal_dates = ['11/30/22', '12/31/22', '1/31/23']
injection_rate = 1000  # units per month
withdrawal_rate = 1000
max_storage_volume = 3000
storage_cost_per_month = 500  # fixed monthly cost

# Run the pricing model
contract_value = price_storage_contract(
    price_data,
    injection_dates,
    withdrawal_dates,
    injection_rate,
    withdrawal_rate,
    max_storage_volume,
    storage_cost_per_month
)

print("Contract Value:", contract_value)

Contract Value: -2500.0


C:\Users\dolly\AppData\Local\Temp\ipykernel_14420\2562794164.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  price_data['Dates'] = pd.to_datetime(price_data['Dates'])


In [ ]:
# Sample Input 2
injection_dates = ['4/30/23', '5/31/23', '6/30/23', '7/31/23']
withdrawal_dates = ['2/29/24', '3/31/24', '4/30/24']
injection_rate = 800  
withdrawal_rate = 1000
max_storage_volume = 2500
storage_cost_per_month = 600 

# Run the pricing model again
contract_value = price_storage_contract(
    price_data,
    injection_dates,
    withdrawal_dates,
    injection_rate,
    withdrawal_rate,
    max_storage_volume,
    storage_cost_per_month
)

print("Contract Value:", contract_value)

Contract Value: -1070.0
